# Zero-Shot Prompting

Zero-shot prompting refers to a technique in prompt engineering where you provide a model with a task without any prior examples. The model is expected to understand and generate a response or complete the task purely based on the given instruction.

In other words, the model is given "zero" prior training examples or demonstrations in the prompt and relies on its pre-trained knowledge to infer what is needed.

## References:
* [Wei et al. (2022)](https://arxiv.org/pdf/2109.01652.pdf): demonstrate how instruction tuning improves zero-shot learning 

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fzero_shot.ipynb)



In [19]:
############################################################
## ZERO-SHOT PROMPT: PROJECT OVERVIEW
############################################################
import os

os.makedirs("data", exist_ok=True)

import csv
from datetime import datetime
from _pipeline import create_payload, model_req

# 1) Define the zero-shot prompt (no examples)
ZERO_SHOT_PROMPT = """You are an advanced prompt-based report skeleton generator. Your primary function is to transform raw, unstructured input text into a well-structured project report skeleton using Markdown. Your response should meticulously extract the Project Title and Problem Statement, ensuring their clarity and prominence. Additionally, you must organize the report in a logical flow with comprehensive placeholders, making it suitable for professional documentation.

The skeleton should include, but is not limited to, the following sections:

1. **Project Overview**

   - **Project Title:** Automatically extract and display the title prominently.
   - **Problem Statement:** Clearly summarize the core problem the project aims to address.
   - **Goal:** Define the primary goal of the project in a precise manner.
   - **Key Objectives:** Enumerate specific, measurable objectives essential to the project.
   - **Scope:** Explicitly outline the boundaries and limitations of the project.

2. **Methodology**

   - **Data & Methods:** Provide placeholders for detailed explanations of the methodologies employed and data sources used.
   - **Code Notebooks:** Include placeholders for structured code cells where necessary.
     ```python
     # Example Notebook Code Block
     # Initialize your data analysis pipeline here
     ```

3. **Results & Evaluation**

   - **Results Overview:** Allocate space for documenting key findings and insights.
   - **Evaluation Metrics:** Define placeholders for performance metrics and assessment criteria.

4. **Discussion & Conclusion**

   - **Discussion:** Introduce a structured discussion section for interpretation of results and possible limitations.
   - **Conclusion:** Summarize key takeaways and propose potential future work directions.

5. **Appendix/Additional Resources**

   - **Supporting Code and Data:** Include necessary supplementary materials such as additional code snippets and references.
     ```python
     # Additional analysis or visualization code
     ```

### Guidelines:

- **Extract Information Accurately:** Ensure that extracted elements, such as the project title and problem statement, are clearly defined and well-formatted.
- **Comprehensive Detailing:** Provide a structured step-by-step report framework that follows best documentation practices.
- **Consistent Markdown Formatting:** Maintain Markdown syntax to ensure a professional and readable report structure.
- **Enhance Clarity:** Utilize concise and informative placeholder text to maintain coherence and usability.

## Example Output Skeleton:

# Report Skeleton

## Project Overview

- **Project Title:** [Extracted Project Title]
- **Problem Statement:** [Extracted Problem Statement]
- **Goal:** [Clearly defined project goal]
- **Key Objectives:**
  - Objective 1
  - Objective 2
  - Objective 3
- **Scope:** [Define the scope and constraints of the project]

## Methodology

- **Data & Methods:** [Detailed methodology description]
- **Notebook Code Blocks:**
  ```python
  # Code for preprocessing and analysis
  ```


"""

# 2) Prepare the payload (choose model & parameters)
payload = create_payload(
    target="open-webui",
    model="gemma2",
    prompt=ZERO_SHOT_PROMPT,
    temperature=1.2,      # Slightly higher => more creative language
    max_tokens=300,       # Adjust if you see truncated output
    num_ctx=100,          # Enough context for a short prompt
    num_predict=200       # Sufficient tokens for a robust overview
)

# Dummy variable to send a message
dummy_message = "research on financial literacy using machine learning."
payload['messages'][0]['content'] += f"\n\n{dummy_message}"

# 3) Send request and capture the response
time_taken, zero_shot_response = model_req(payload=payload)

# 4) Display the raw output and timing
print("===== ZERO-SHOT OUTPUT =====")
print(zero_shot_response)
if time_taken:
    print(f"\nTime taken: {time_taken}s")

# Save the zero-shot response to a variable for use in the few-shot notebook
with open("data/zero_shot_output.txt", "w", encoding="utf-8") as f:
    f.write(zero_shot_response)

# 5) (Optional) Log the result for future reference
#    This helps if you want to compare with Few-Shot or Chain-of-Thought later.
log_entry = [
    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "zero_shot",
    "gemma2",
    1.2,
    time_taken,
    zero_shot_response.replace("\n", "\\n")  # escape newlines for CSV
]

with open("data/zero_shot_logs.csv", "a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    # If it's a brand-new file, you could write a header row here
    writer.writerow(log_entry)


Payload: {'model': 'gemma2', 'messages': [{'role': 'user', 'content': 'You are an advanced prompt-based report skeleton generator. Your primary function is to transform raw, unstructured input text into a well-structured project report skeleton using Markdown. Your response should meticulously extract the Project Title and Problem Statement, ensuring their clarity and prominence. Additionally, you must organize the report in a logical flow with comprehensive placeholders, making it suitable for professional documentation.\n\nThe skeleton should include, but is not limited to, the following sections:\n\n1. **Project Overview**\n\n   - **Project Title:** Automatically extract and display the title prominently.\n   - **Problem Statement:** Clearly summarize the core problem the project aims to address.\n   - **Goal:** Define the primary goal of the project in a precise manner.\n   - **Key Objectives:** Enumerate specific, measurable objectives essential to the project.\n   - **Scope:** Ex

---

## How to improve it?

* **Use Clear and Concise Instructions**: Be specific about the task and desired format.
    * Bad Prompt: “Summarize this.”
    * Good Prompt: “Summarize this paragraph in one sentence.”
* **Add Context**: Providing background can help the model interpret ambiguous prompts better.
* **Specify Output Format**: If a particular structure is needed, describe it in the instruction.